In [1]:
import numpy as np
from hello.pid.lvpid import PIDController
from hello.pid.delay import seconds, minutes, hours, days, m2s, s2m, h2s, DelayBuffer, DelaySink
from hello.pid.gas_process import HeadspaceProcess, GasController
from hello.pid.do_simulation.doprocess import DOProcess, AIR_CNO

from hello.pid.do_simulation.options import SimOps, SimConfig
from hello.pid.do_simulation.fitness import Fitness2, Fitness3
from hello.pid.picker import mk_picker
from hello.pid.do_simulation.sim_window import PIDSimFrame
from hello.pid.ui import TkQuitHack

### Copy this template to ensure all sim options are set correctly

In [2]:
ops = SimOps()
ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 1
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = 5
ops.n2_pid.i = 5
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta = 1
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 10
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.plots.xscale = 'auto'
ops.plots.xmin = 0
ops.plots.xmax = 72
ops.plots.xscale_factor = 3600

ops.delay = 0
ops.end = 10000
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 90
ops.set_point = 40
ops.set_point_deadband = 1
ops.k_mult = 1.1
ops.k = None
ops.c = None
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55
ops.time_unit = hours
ops.max_iters = 3 * days

In [80]:
cfg = SimConfig()
cfg.simops = ops
def run3():
    global root, f
    cfg.simops.o2_pid.p = 7
    cfg.simops.o2_pid.i = 50
    cfg.simops.o2_pid.amax = 100
    cfg.simops.o2_pid.deadband = 0
    cfg.simops.o2_pid.beta = 0
    cfg.simops.o2_pid.man_request = 10

    cfg.simops.n2_pid.p = -2
    cfg.simops.n2_pid.i = 60
    cfg.simops.n2_pid.amax = 90
    cfg.simops.n2_pid.deadband = 0
    cfg.simops.n2_pid.beta = 0
    cfg.simops.n2_pid.man_request = 10
    cfg.simops.delay=0*minutes

    cfg.update_interval = 50
    cfg.time_factor = 200

    cfg.simops.initial_pv = 100
    cfg.simops.set_point = 100

    cfg.xwindow_hrs = 10

    root = TkQuitHack()
    root.wm_title("DO Simulation")
    #root.geometry("%dx%d"%(600, 800))
    f = PIDSimFrame(root, cfg)
    f.pack()
    root.mainloop()
run3()

State Value Updated: sp 1.0 
State Value Updated: sp 100.0 
State Value Updated: sp 100.0 
Value Changed: o2_pid.pgain: 10.00
Value Changed: o2_pid.itime: 80.00
Value Changed: o2_pid.dtime: 0.00
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
Advancing simulation 3600 seconds...
TIMEFACTOR_CHANGED : 0.0
State Value Updated: sp 150.0 
TIMEFACTOR_CHANGED : 200.0
Value Changed: n2_pid.pgain: -4.00
State Value Updated: sp 100.0 
Advancing simulation 36000 seconds...
State Value Updated: sp 150.0 


In [3]:
from hello.pid.do_simulation.do_sim import do_sim_coroutine, do_sim

In [4]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
    return f
def get_text(ops):

    text = \
"""
O2: P=%3d  I=%5d  Initial PV: %3d  k: %.4f/hr (%sx Est.)
N2: P=%3d  I=%4d  Set Point:  %3d  c: %s%%/hr

 """ % (ops.o2_pid.p, ops.o2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.n2_pid.p, ops.n2_pid.i, ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text

In [5]:
def run(ops):
    global data, data2, text, data3
    data, data2, data3 = do_sim(ops)
    text = get_text(ops)
    plot(ops)
    
def run2(ops):
    global data, data2, text, data3
    data, data2, data3 = do_sim(ops)
    text = get_text(ops)
    plot2(ops)

In [6]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, cs, co2_req, n2_req, o2_req, air_req = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=ops.set_point, ls="--", color="black")

    ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")
    ax3.plot(xs, ud[::step], "purple", ls="-", label="Ud")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    if np.min(pvs) - 1 < m:
        m -= 1
    if np.max(pvs) + 1 > ma:
        ma += 1
    ax1.set_ylim((m, ma))
    #ax1.set_ylim((sp-5, sp+5))


    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

Using matplotlib backend: TkAgg


In [7]:
# Same as plot() but doesn't have the PID internal values plot

def plot2(ops):    
    print("Parsing Data")
    global ax1, ax2, fig
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, cs, co2_req, n2_req, o2_req, air_req = list(zip(*data))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=ops.set_point, ls="--", color="black")

    ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    if np.min(pvs) - 1 < m:
        m -= 1
    if np.max(pvs) + 1 > ma:
        ma += 1
    ax1.set_ylim((m, ma))

    for a in ax1, ax2: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

In [8]:
k_80L = 0.1306
k_15L = 0.19314

In [98]:
ops = SimOps()
ops.o2_pid.p = 10
ops.o2_pid.i = 80
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 0
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = -4
ops.n2_pid.i = 60
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta = 0
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 2
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.plots.xscale = 'auto'
ops.plots.xmin = 0
ops.plots.xmax = 72
ops.plots.xscale_factor = 3600

ops.delay = 0
ops.end = 16*hours
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = 0,0,0
ops.initial_pv = 100
ops.set_point = 150
ops.set_point_deadband = 1
ops.k_mult = 1.1
ops.k = 0.1306
ops.c = 0
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 57
ops.time_unit = hours
ops.max_iters = 3 * days

run2(ops)

from matplotlib.ticker import MultipleLocator
l = MultipleLocator(2)
for a in plt.gcf().axes:
    a.xaxis.set_major_locator(l)

#ax1.set_ylim(45, 105)
ax2.set_ylim(0, 1)

Parsing Data
Closing Plot
Plotting Data


(0, 1)

In [ ]:
from hello.pid.do_simulation.fitness import Fitness2, Fitness3
from hello.pid.picker import mk_picker

In [49]:
%matplotlib
ops.end = 10 * hours
sp = 150

ops.o2_pid.p = 1
ops.o2_pid.i = 50
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 0
ops.o2_pid.d = 0
ops.o2_pid.alpha = -1

ops.n2_pid.p = -0.5
ops.n2_pid.i = 60
ops.n2_pid.amax = 100
ops.n2_pid.deadband = 0
ops.n2_pid.d = 0
ops.n2_pid.alpha = -1

ops.initial_pv = 100
ops.set_point = sp
ops.c = 0
ops.dc = 0
ops.d2c = 0

# k_mult = 1.25
ops.k_mult = 1.1
ops.k = 0.1306
ops.initial_actual_cno = (0.0000, .78, .2095)
ops.initial_request_cno = (0.00, 0, 0)
ops.mode = 'm2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 1
ops.delay = 0

results = []
f = FuncFormatter(lambda y, _: "%.1f%%"%y)
plt.close()
fig = plt.figure()
ax = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)
ax.axhline(y=sp, ls="--", color="black")
ax.grid()

ax.yaxis.set_major_formatter(f)
ax2.yaxis.set_major_formatter(f)
ax3.yaxis.set_major_formatter(f)

def plot3(x, pv, o2_req, n2_req, label):
    ax.plot(x, pv, label=label)
    ax2.plot(x, np.array(o2_req)*100, label=label)
    ax3.plot(x, np.array(n2_req)*100, label=label)

combos = [(p,i) for p in range(1,2) for i in range(50, 101, 10)]
    
for p, i in combos:
    if p is not None:
        ops.o2_pid.p = p
    if i is not None:
        ops.o2_pid.i = i
    d1, d2, d3 = do_sim(ops)
    x, pv, cs, co2_req, n2_req, o2_req, air_req = list(zip(*d1))
    plot3(x, pv, o2_req, n2_req, "P:%.1f I:%.1f"%(p,i))
    
    x = np.array(x)
    pv = np.array(pv)
    ind = peakutils.indexes(pv, thres=0.5, min_dist = 5*minutes)
    ax.plot(x[ind], pv[ind], '+', mfc=None, mec='r', mew=2, ms=8)
    print(p,i, str(Fitness3(x,pv,150)), ind/3600, pv[ind])

for a in (ax, ax2, ax3):
    b = a.get_position()
    a.set_position([b.x0, b.y0, b.width*0.9, b.height])
    a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left", fontsize=10)
    mk_picker(fig, a)

Using matplotlib backend: TkAgg


NameError: name 'do_sim2' is not defined

In [56]:
def sln_space_test(ops):
    d1, d2, d3 = do_sim(ops)
    x, pv, cs, co2_req, n2_req, o2_req, air_req = [np.array(d) for d in zip(*d1)]
    cs = cs
    co2_req = co2_req
    n2_req = n2_req
    o2_req = o2_req
    air_req = air_req
    del d1, d2, d3  # garbage collect
    score = Fitness3(x, pv, ops.set_point)
    score2 = Fitness2((x, pv, ops.set_point))
    return locals().copy()

In [100]:
plot_test_results.clear()

In [103]:
from mpl_toolkits.mplot3d import Axes3D
leg_map = {}

sp = 150

try:
    plot_test_results
except NameError:
    plot_test_results = {}

def on_legend_clicked(event):
    ll = event.artist
    line = leg_map[ll]
    line.set_visible(not line.get_visible())
    fig.canvas.draw()
fig.canvas.mpl_connect("pick_event", on_legend_clicked)


def plot2(x, pv, o2_req, n2_req, label, update=True):
    l1 = ax.plot(x, pv, label=label)
    ax.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left", fontsize=10)
    if update:
        fig.canvas.draw()
        fig.canvas.flush_events()
    
scatter_data = [[],[],[]]
def plot3(p, i, score, update=True):
    x,y,z = scatter_data
    x.append(p)
    y.append(i)
    z.append(score)
    ax3d.clear()
    ax3d.scatter(x,y,z, antialiased=False)
    if update:
        fig.canvas.draw()
        fig.canvas.flush_events()

#plt.close()
fig = plt.figure()
ax = fig.add_subplot(2,1,1)
ax3d = fig.add_subplot(2,1,2, projection='3d')
ax.axhline(y=sp, ls="--", color="black")
ax.grid()
ax.axhline(y=sp+3, color="black", ls="--")
ax.axhline(y=sp-3, color="black", ls="--")
#ax.set_ylim(sp-10, sp+10)

f = FuncFormatter(lambda y, _: "%.1f%%"%y)
ax.yaxis.set_major_formatter(f)

ops.o2_pid.p = 10
ops.o2_pid.i = 80
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 0
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = -4
ops.n2_pid.i = 60
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta=0
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 1
ops.mfcs.n2_max = 10
ops.mfcs.o2_max = 2
ops.mfcs.air_max = 10

ops.initial_pv = 100
ops.set_point = sp
ops.mode="o2a"

ops.end = 16*hours
ops.set_point_deadband = 1
ops.delay = 0
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = (0, 0, 0)
ops.k_mult = 1.1
ops.k = 0.1306
ops.reactor_size = 80
ops.reactor_volume = 57

ops.dc = 0
ops.d2c = 0

ops.main_gas = 1.0
ops.time_unit = hours
ops.max_iters = 24 * days

axes = [ax, ax3d]

for a in axes:
    b = a.get_position()
    a.set_position([b.x0, b.y0, b.width*0.9, b.height])

passing = []
update = True
noupdate = 0
Fitness2.max_overshoot = 3
Fitness2.settle_target = 5 * hours

pid=ops.o2_pid

for p in range(6, 12,2):
    for i in range(50,160, 10):
        print("\rTesting P:%d I:%d " % (p,i), end="")
        if (p,i) not in plot_test_results:
            pid.p = p
            pid.i = i
            data = sln_space_test(ops)
            update = True
        else:
            data = plot_test_results[(p,i)]
            update = False
            noupdate += 1
            if noupdate > 10:
                noupdate = 0
                update = True
        s2 = Fitness2((data['x'], data['pv'], sp))
        res, reason = s2.result_reason()
        if res == True:
            passing.append((p,i))
            label = "P:%3d I:%3d" % (p, i)
            plot2(data['x'][::120], data['pv'][::120],data['o2_req'], data['n2_req'], label, update)
            plot_test_results[(p,i)] = data
        plot3(p,i, data['score'].score(), update)
        print("Result: %s %s %s" % (str(data['score']), res, reason))
        fig.canvas.flush_events()
print("Done")

Testing P:6 I:50 Result: Total:16.25 Peak:1.56 Settle:3.72 True 
Testing P:6 I:60 Result: Total:15.13 Peak:1.11 Settle:3.73 True 
Testing P:6 I:70 Result: Total:14.99 Peak:0.82 Settle:3.78 True 
Testing P:6 I:80 Result: Total:15.65 Peak:0.80 Settle:3.87 True 
Testing P:6 I:90 Result: Total:18.18 Peak:0.72 Settle:4.20 True 
Testing P:6 I:100 Result: Total:22.32 Peak:0.99 Settle:4.62 True 
Testing P:6 I:110 Result: Total:22.43 Peak:-2.00 Settle:5.14 False Settle 18508 > 18000 
Testing P:6 I:120 Result: Total:33.11 Peak:-2.00 Settle:6.09 False Settle 21931 > 18000 
Testing P:6 I:130 Result: Total:40.26 Peak:-2.00 Settle:6.65 False Settle 23950 > 18000 
Testing P:6 I:140 Result: Total:46.90 Peak:-2.00 Settle:7.13 False Settle 25684 > 18000 
Testing P:6 I:150 Result: Total:54.48 Peak:-2.00 Settle:7.65 False Settle 27529 > 18000 
Testing P:8 I:50 Result: Total:12.71 Peak:1.21 Settle:3.35 True 
Testing P:8 I:60 Result: Total:12.35 Peak:0.92 Settle:3.39 True 
Testing P:8 I:70 Result: Total:12.

KeyboardInterrupt: 

In [104]:
mk_picker(fig, ax)

In [19]:
try:
    _g_leg_map
except NameError:
    _g_leg_map = {}
def plot4(pd, sp):
    global fig, ax, ax2, ax3
    _g_leg_map.clear()
    mp = {}
    if not plt.get_fignums():
        fig = plt.figure()
        new = True
        ax = fig.add_subplot(311)
        ax2 = fig.add_subplot(3,1,2)
        ax3 = fig.add_subplot(3,1,3)
    else:
        new = False
        ax.clear()
        ax2.clear()
        ax3.clear()

    f = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
    ax2.yaxis.set_major_formatter(f)
    ax3.yaxis.set_major_formatter(f)
    print(*("%-12s"%a for a in "p i settle osc".split()))
    def _p(*args):
        print(*("%-12.1f"%a for a in args))
    for (p,i), (x,pv,n2,o2) in pd.items():
        f =  Fitness2()
        f.settle_target = 5 * hours
        f.calculate((x,pv,50))
        s,o = f.result2()
        _p(p,i, s/3600, o)
        label = "P:%d I:%d"%(p,i)
        ax.plot(x,pv, label=label + " PV")
        ax2.plot(x, n2, label=label+" N2")
        ax3.plot(x, o2, label=label+" O2")
    for a in (ax, ax2, ax3):
        if new:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.80, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        mk_picker(fig, a)
    ax.axhline(y=sp-1, ls="--", color="black")
    ax.axhline(y=sp+1, ls="--", color="black")
    ax.set_ylim(sp-5, sp+5)
    #ax2.set_ylim(0, 110)
    
def test3(ops):
    d1, d2, d3 = do_sim(ops)
    x, pv, _, _, n2, o2, *_ = list(zip(*d1))
    return x,pv, n2, o2
    

In [ ]:
plot4(pd, sp)

# 15L DO Simulations for Results Report

In [106]:
ops = SimOps()
ops.o2_pid.p = 7
ops.o2_pid.i = 50
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 0
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = -6
ops.n2_pid.i = 90
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta = 0
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = .1
ops.mfcs.o2_max = 0.5
ops.mfcs.n2_max = 2
ops.mfcs.air_max = 2

ops.plots.xscale = 'auto'
ops.plots.xmin = 0
ops.plots.xmax = 72
ops.plots.xscale_factor = 3600

ops.delay = 0
ops.end = 20*hours
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = (0.00, 0, 0)
ops.initial_pv = 100
ops.set_point = 50
ops.set_point_deadband = 1
ops.k_mult = 1.0
ops.k = k_15L
ops.c = 0
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 15
ops.reactor_volume = 11.5
ops.time_unit = hours
ops.max_iters = 3 * days

In [37]:
from matplotlib.ticker import MultipleLocator
pd = {}
sp = 50
p = 7
i = 50
pd[(p,i)] = test3(ops)
plot4(pd, sp)
for a in (ax, ax2, ax3):
    a.grid()
    l = MultipleLocator(2)
    a.xaxis.set_major_locator(l)

p            i            settle       osc         
7.0          50.0         -0.8         1.0         


### Graph generation for Results report for 100% to 150% ramp

In [79]:
ops.set_point = 150
ops.mfcs.o2_max = 2
x, pv, n2, o2 = test3(ops)
plt.close()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax2 = ax.twinx()
f = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
l = MultipleLocator(2)
ax.xaxis.set_major_locator(l)
ax2.yaxis.set_major_formatter(f)
ax.plot(x,pv, "blue", ls="-", label="PV")
ax2.plot(x,o2, "green", ls="-", label="O2 Flow")
ax2.plot(x,n2, "red", ls="-", label="N2 Flow")
ax.xaxis.set_label_text("Time(hr)")
ax.yaxis.set_label_text("DOPV(%)")
ax2.yaxis.set_label_text("Gas Flow Request (%)")
ax.grid()
ax.set_ylim(100, 160)
ax2.set_ylim(0, .80)
fig.text(0.15, 0.96, get_text(ops), transform=ax.transAxes, fontsize=12,
        verticalalignment='top')
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
h1.extend(h2)
l1.extend(l2)
ax.legend(h1, l1, loc="center", ncol=3, bbox_to_anchor=(0.5, -.15))
b = ax.get_position()
ax.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))
ax2.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))

### Graph generation for Results report for 100% to 50% ramp

In [67]:
from matplotlib.ticker import MultipleLocator
ops.k_mult = 0.9
ops.set_point = 50

ops.n2_pid.p = -3
x, pv, n2, o2 = test3(ops)
plt.close()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax2 = ax.twinx()
f = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
ax2.yaxis.set_major_formatter(f)
l = MultipleLocator(2)
ax.xaxis.set_major_locator(l)
ax.plot(x,pv, "blue", ls="-", label="PV")
ax2.plot(x,o2, "green", ls="-", label="O2 Flow")
ax2.plot(x,n2, "red", ls="-", label="N2 Flow")
ax.xaxis.set_label_text("Time(hr)")
ax.yaxis.set_label_text("DOPV(%)")
ax2.yaxis.set_label_text("Gas Flow Request (%)")
ax.grid()
ax.set_ylim(40, 110)
ax2.set_ylim(0, 1.00)
fig.text(0.15, 0.96, get_text(ops), transform=ax.transAxes, fontsize=12,
        verticalalignment='top')
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
h1.extend(h2)
l1.extend(l2)
ax.legend(h1, l1, loc="center", ncol=3, bbox_to_anchor=(0.5, -.15))
b = ax.get_position()
ax.set_position((b.x0, b.y0+0.05, b.width*0.95,b.height*0.9))
ax2.set_position((b.x0, b.y0+0.05, b.width*0.95,b.height*0.9))

### Comparison of effect of Main Gas on controller performance

In [100]:
ops.set_point = 150


plt.close()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

l = MultipleLocator(2)
ax.xaxis.set_major_locator(l)
ax.xaxis.set_label_text("Time(hr)")
ax.yaxis.set_label_text("DOPV(%)")
ax.grid()
ax.set_ylim(100, 160)
fig.text(0.15, 0.96, get_text(ops), transform=ax.transAxes, fontsize=12,
        verticalalignment='top')
for mg in (0.1, 0.5, 1.0, 2.0):
    ops.main_gas = mg
    x, pv, n2, o2 = test3(ops)
    ax.plot(x,pv, label="%.1f LPM"%mg)
    
    
    
b = ax.get_position()
ax.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))
ax.legend(loc="center", ncol=4, bbox_to_anchor=(0.5, -.15))

In [109]:
ops.set_point = 150

plt.close()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

l = MultipleLocator(2)
ax.xaxis.set_major_locator(l)
ax.xaxis.set_label_text("Time(hr)")
ax.yaxis.set_label_text("DOPV(%)")
ax.grid()
ax.set_ylim(100, 160)
fig.text(0.15, 0.96, get_text(ops), transform=ax.transAxes, fontsize=12,
        verticalalignment='top')

ops.reactor_volume = 11
ops.main_gas = 2
x, pv, n2, o2 = test3(ops)
ax.plot(x,pv, label="2 LPM")
    
ops.reactor_volume = 13
ops.main_gas = 1
x, pv, n2, o2 = test3(ops)
ax.plot(x,pv, label="1 LPM"%mg)
    
b = ax.get_position()
ax.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))
ax.legend(loc="center", ncol=4, bbox_to_anchor=(0.5, -.15))

In [101]:
ax.set_ylim(140, 160)

(140, 160)

# 80L Simulations for Results Report

In [90]:
ops = SimOps()
ops.o2_pid.p = 6
ops.o2_pid.i = 50
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 0
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = -4
ops.n2_pid.i = 60
ops.n2_pid.d = 0
ops.n2_pid.amax = 100
ops.n2_pid.amin = 0
ops.n2_pid.beta = 0
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 10
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.plots.xscale = 'auto'
ops.plots.xmin = 0
ops.plots.xmax = 72
ops.plots.xscale_factor = 3600

ops.delay = 0
ops.end = 20*hours
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = (0.00, 0, 0)
ops.initial_pv = 100
ops.set_point = 50
ops.set_point_deadband = 1
ops.k_mult = 1.1
ops.k = k_80L
ops.c = 0
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 10.0
ops.reactor_size = 80
ops.reactor_volume = 57
ops.time_unit = hours
ops.max_iters = 3 * days

In [91]:
ops.set_point = 150
x, pv, n2, o2 = test3(ops)
plt.close()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax2 = ax.twinx()
f = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
l = MultipleLocator(2)
ax.xaxis.set_major_locator(l)
ax2.yaxis.set_major_formatter(f)
ax.plot(x,pv, "blue", ls="-", label="PV")
ax2.plot(x,o2, "green", ls="-", label="O2 Flow")
ax2.plot(x,n2, "red", ls="-", label="N2 Flow")
ax.xaxis.set_label_text("Time(hr)")
ax.yaxis.set_label_text("DOPV(%)")
ax2.yaxis.set_label_text("Gas Flow Request (%)")
ax.grid()
ax.set_ylim(100, 160)
ax2.set_ylim(0, .80)
fig.text(0.15, 0.96, get_text(ops), transform=ax.transAxes, fontsize=12,
        verticalalignment='top')
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
h1.extend(h2)
l1.extend(l2)
ax.legend(h1, l1, loc="center", ncol=3, bbox_to_anchor=(0.5, -.15))
b = ax.get_position()
ax.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))
ax2.set_position((b.x0, b.y0+0.05, b.width,b.height*0.9))

In [ ]:
# Maximum value of c that O2 controller can sustain for a given k
ops.k*ops.k_mult*(100/20.95)*100

In [68]:
from officelib.xllib import *
xl = Excel()

In [73]:
_

5.161111111047471

In [75]:
0.16*60

9.6

In [ ]:
#f.w._current_state['o2_pid'].b